# Working with DataFrames

In [ ]:
We have 3 ways to create DFs:
    - Programmatically specify the schema of the DataFrame and manually enter the data in it.
    - From an existing RDD object in Spark.
    - Directly from a data source.

1- Programmatically specify the schema of the DataFrame and manually enter the data in it

In [1]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

from pyspark.sql import *

In [2]:
na_schema = Row("Name","Age")

In [3]:
row1 = na_schema("Juan", 12)
row2 = na_schema("Luz",13)
row3 = na_schema("Diana",14)

In [4]:
na_list = [row1,row2,row3]
type(na_list)


list

In [5]:
df_na = sqlc.createDataFrame(na_list)
type(df_na)

pyspark.sql.dataframe.DataFrame

In [6]:
df_na.show()

+-----+---+
| Name|Age|
+-----+---+
| Juan| 12|
|  Luz| 13|
|Diana| 14|
+-----+---+



2- From an existing RDD object in Spark.

In [7]:
data = [("Juan", 12),("Luz",13),("Diana",14)]
data_rdd = sc.parallelize(data)
type(data_rdd)

pyspark.rdd.RDD

In [8]:
data_df = sqlc.createDataFrame(data_rdd)
type(data_df)

pyspark.sql.dataframe.DataFrame

In [9]:
data_df.show()

+-----+---+
|   _1| _2|
+-----+---+
| Juan| 12|
|  Luz| 13|
|Diana| 14|
+-----+---+



3- Directly from a data source.

In [10]:
df = sqlc.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('iris.csv')
type(df)

pyspark.sql.dataframe.DataFrame

In [11]:
df.show(4)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 4 rows



Looking the DF schema

In [12]:
df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- species: string (nullable = true)



In [13]:
df.schema.names

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

In [14]:
## number of rows
df.count()

150

In [15]:
## number of columns
len(df.columns)

5

In [16]:
## get statistics of all the columns
df.describe().show()

+-------+------------------+-------------------+------------------+------------------+---------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  species|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|     null|
| stddev|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|     null|
|    min|               4.3|                2.0|               1.0|               0.1|   setosa|
|    max|               7.9|                4.4|               6.9|               2.5|virginica|
+-------+------------------+-------------------+------------------+------------------+---------+



In [17]:
## get statistics of individual column
df.describe('sepal_length').show()

+-------+------------------+
|summary|      sepal_length|
+-------+------------------+
|  count|               150|
|   mean| 5.843333333333335|
| stddev|0.8280661279778637|
|    min|               4.3|
|    max|               7.9|
+-------+------------------+



In [18]:
# renaming columns form the DataFrame
df = df.withColumnRenamed('sepal_width','SepalWidth')
df.show(4)

+------------+----------+------------+-----------+-------+
|sepal_length|SepalWidth|petal_length|petal_width|species|
+------------+----------+------------+-----------+-------+
|         5.1|       3.5|         1.4|        0.2| setosa|
|         4.9|       3.0|         1.4|        0.2| setosa|
|         4.7|       3.2|         1.3|        0.2| setosa|
|         4.6|       3.1|         1.5|        0.2| setosa|
+------------+----------+------------+-----------+-------+
only showing top 4 rows



In [19]:
# selection columns
df.select('SepalWidth','sepal_length').show(4)

+----------+------------+
|SepalWidth|sepal_length|
+----------+------------+
|       3.5|         5.1|
|       3.0|         4.9|
|       3.2|         4.7|
|       3.1|         4.6|
+----------+------------+
only showing top 4 rows



Adding and Removing Columns

In [20]:
# Adding a column
df = df.withColumn('half_sepal_width', df['SepalWidth']/2.0)
df.show(4)

+------------+----------+------------+-----------+-------+----------------+
|sepal_length|SepalWidth|petal_length|petal_width|species|half_sepal_width|
+------------+----------+------------+-----------+-------+----------------+
|         5.1|       3.5|         1.4|        0.2| setosa|            1.75|
|         4.9|       3.0|         1.4|        0.2| setosa|             1.5|
|         4.7|       3.2|         1.3|        0.2| setosa|             1.6|
|         4.6|       3.1|         1.5|        0.2| setosa|            1.55|
+------------+----------+------------+-----------+-------+----------------+
only showing top 4 rows



In [21]:
# Removing a column
df = df.drop('half_sepal_width')
df.show(4)

+------------+----------+------------+-----------+-------+
|sepal_length|SepalWidth|petal_length|petal_width|species|
+------------+----------+------------+-----------+-------+
|         5.1|       3.5|         1.4|        0.2| setosa|
|         4.9|       3.0|         1.4|        0.2| setosa|
|         4.7|       3.2|         1.3|        0.2| setosa|
|         4.6|       3.1|         1.5|        0.2| setosa|
+------------+----------+------------+-----------+-------+
only showing top 4 rows



Distinct Values

In [22]:
df.select('species').distinct().show()

+----------+
|   species|
+----------+
| virginica|
|versicolor|
|    setosa|
+----------+



Removing Duplicate Rows and Fitering

In [23]:
df.select('species').dropDuplicates().show()

+----------+
|   species|
+----------+
| virginica|
|versicolor|
|    setosa|
+----------+



In [24]:
df.filter(df.species == 'setosa').show(4)

+------------+----------+------------+-----------+-------+
|sepal_length|SepalWidth|petal_length|petal_width|species|
+------------+----------+------------+-----------+-------+
|         5.1|       3.5|         1.4|        0.2| setosa|
|         4.9|       3.0|         1.4|        0.2| setosa|
|         4.7|       3.2|         1.3|        0.2| setosa|
|         4.6|       3.1|         1.5|        0.2| setosa|
+------------+----------+------------+-----------+-------+
only showing top 4 rows



In [25]:
df.filter((df.sepal_length > 5)&(df.species == 'setosa')).show(4)

+------------+----------+------------+-----------+-------+
|sepal_length|SepalWidth|petal_length|petal_width|species|
+------------+----------+------------+-----------+-------+
|         5.1|       3.5|         1.4|        0.2| setosa|
|         5.4|       3.9|         1.7|        0.4| setosa|
|         5.4|       3.7|         1.5|        0.2| setosa|
|         5.8|       4.0|         1.2|        0.2| setosa|
+------------+----------+------------+-----------+-------+
only showing top 4 rows



Ordering

In [26]:
## by default is ascending
df.orderBy(df.sepal_length).show(5)

+------------+----------+------------+-----------+-------+
|sepal_length|SepalWidth|petal_length|petal_width|species|
+------------+----------+------------+-----------+-------+
|         4.3|       3.0|         1.1|        0.1| setosa|
|         4.4|       3.2|         1.3|        0.2| setosa|
|         4.4|       3.0|         1.3|        0.2| setosa|
|         4.4|       2.9|         1.4|        0.2| setosa|
|         4.5|       2.3|         1.3|        0.3| setosa|
+------------+----------+------------+-----------+-------+
only showing top 5 rows



In [27]:
## descending
df.orderBy(df.sepal_length.desc()).show(5)

+------------+----------+------------+-----------+---------+
|sepal_length|SepalWidth|petal_length|petal_width|  species|
+------------+----------+------------+-----------+---------+
|         7.9|       3.8|         6.4|        2.0|virginica|
|         7.7|       2.6|         6.9|        2.3|virginica|
|         7.7|       3.8|         6.7|        2.2|virginica|
|         7.7|       3.0|         6.1|        2.3|virginica|
|         7.7|       2.8|         6.7|        2.0|virginica|
+------------+----------+------------+-----------+---------+
only showing top 5 rows



Aggregating Values

In [28]:
df.groupBy('species').agg({'SepalWidth':'mean'}).show()

+----------+------------------+
|   species|   avg(SepalWidth)|
+----------+------------------+
| virginica|2.9739999999999998|
|versicolor|2.7700000000000005|
|    setosa|3.4180000000000006|
+----------+------------------+



In [29]:
df.groupBy('species').count().show()

+----------+-----+
|   species|count|
+----------+-----+
| virginica|   50|
|versicolor|   50|
|    setosa|   50|
+----------+-----+

